<a href="https://colab.research.google.com/github/ferdikurnazdm/MegaChatBot/blob/main/01_20240711_CHAT_BOT_WORKSPACE_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#01_20240530_GEREKLI_KUTUPHANELER
!pip uninstall pandas -y
!pip uninstall nltk -y
!pip uninstall tensorflow -y
!pip uninstall keras -y
!pip uninstall tf2onnx -y
!pip install pandas==2.0.3
!pip install nltk
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install tf2onnx

In [ ]:
#02_20240530_MODEL_TRAINING_CODE_V1 [OBSOLITE]
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tag import sequential

lemmatizer = WordNetLemmatizer()
intents=json.loads(open('/content/intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
print(words)
classes=sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl','wb'))
training=[]
output_empty=[0]*len(classes)
for document in documents:
  bag=[]
  word_pattern=document[0]
  word_pattern=[lemmatizer.lemmatize(word.lower())for word in word_pattern]
  for word in words:
    bag.append(1) if word in word_pattern else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag, output_row])

random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model', hist)
print("done")

In [ ]:
#02_20240530_MODEL_TRAINING_CODE_V2 [OBSOLITE]
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# JSON verilerini yükleyin
intents = json.loads(open('/content/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# JSON dosyasındaki verileri işleyin
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

# Eğitim verilerini NumPy dizisine dönüştürün
training = np.array(training, dtype=object)

train_x = np.array([element[0] for element in training])
train_y = np.array([element[1] for element in training])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# SGD optimizasyonunu yeni yöntemle oluşturun
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model', hist)

print("done")



In [ ]:
#02_20240530_MODEL_TRAINING_CODE_V3
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import legacy as legacy_optimizers

# NLTK veri indirme
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# intents.json dosyasını yükleyin
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# JSON dosyasındaki verileri işleyin
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Eğitim verilerini oluşturun
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Verileri numpy array'e dönüştürün
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]), dtype=np.float32)
train_y = np.array(list(training[:, 1]), dtype=np.float32)

# Modeli oluşturun ve eğitin
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# SGD optimizer'ı legacy optimizers içinden kullanarak tanımlayın
sgd = legacy_optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model', hist)

print("done")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Epoch 1/200
29/29 [==============================] - 1s 2ms/step - loss: 1.9606 - accuracy: 0.1986
Epoch 2/200
29/29 [==============================] - 0s 2ms/step - loss: 1.8116 - accuracy: 0.2908
Epoch 3/200
29/29 [==============================] - 0s 2ms/step - loss: 1.5568 - accuracy: 0.5532
Epoch 4/200
29/29 [==============================] - 0s 2ms/step - loss: 1.2579 - accuracy: 0.5887
Epoch 5/200
29/29 [==============================] - 0s 2ms/step - loss: 0.9053 - accuracy: 0.7163
Epoch 6/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6989 - accuracy: 0.7589
Epoch 7/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6111 - accuracy: 0.8156
Epoch 8/200
29/29 [==============================] - 0s 2ms/step - loss: 0.5114 - accuracy: 0.8369
Epoch 9/200
29/29 [==============================] - 0s 2ms/step - loss: 0.4023 - accuracy: 0.8298
Epoch 10/200
29/29 [==============================] - 0s 2ms/step - loss: 0.4103 - accuracy: 0.8298
Epoch 11/

In [ ]:
#03_20240530_MODEL_USING_CODE_V1
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer=WordNetLemmatizer()
intents=json.loads(open('/content/intents.json').read())

words=pickle.load(open('/content/words.pkl','rb'))
classes=pickle.load(open('/content/classes.pkl','rb'))
model=load_model('/content/chatbot_model.model')
def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word==w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

print("go bot is running")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints, intents)
  print(res)

In [ ]:
#04_20240530_MODEL_TO_ONNX_FORMAT_V1
import tensorflow as tf
import tf2onnx

#Model yükleme
model = tf.keras.models.load_model('chatbot_model.model')

#ONNX formatına dönüştürme
spec = (tf.TensorSpec((None, len(train_x[0])), tf.float32, name="input"),)
output_path = "chatbot_model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())


In [ ]:
#05_20240530_WORDS_AND_CLASSES_EXPORT_AS_JSON_V1
import pickle
import json

# words.pkl ve classes.pkl dosyalarını yükleyin
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

# JSON formatına dönüştürün ve dosyaya yazın
with open("words.json", "w") as f:
    json.dump(words, f)
with open("classes.json", "w") as f:
    json.dump(classes, f)
